<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Topic-Modelling-using-LDA" data-toc-modified-id="Topic-Modelling-using-LDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Topic Modelling using LDA</a></span></li><li><span><a href="#DBOW-MODEL" data-toc-modified-id="DBOW-MODEL-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>DBOW MODEL</a></span><ul class="toc-item"><li><span><a href="#Checking-similarity-vectors-in-DBOW-Model" data-toc-modified-id="Checking-similarity-vectors-in-DBOW-Model-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Checking similarity vectors in DBOW Model</a></span></li></ul></li><li><span><a href="#PV-DMM-(Mean)" data-toc-modified-id="PV-DMM-(Mean)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>PV-DMM (Mean)</a></span><ul class="toc-item"><li><span><a href="#Checking-similarity-vectors-in-PV-DMM-Model" data-toc-modified-id="Checking-similarity-vectors-in-PV-DMM-Model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Checking similarity vectors in PV-DMM Model</a></span></li></ul></li><li><span><a href="#PV-DMC-(concatnation)" data-toc-modified-id="PV-DMC-(concatnation)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>PV-DMC (concatnation)</a></span><ul class="toc-item"><li><span><a href="#Checking-similarity-vectors-in-PV-DMC-Model-(concatenation)" data-toc-modified-id="Checking-similarity-vectors-in-PV-DMC-Model-(concatenation)-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Checking similarity vectors in PV-DMC Model (concatenation)</a></span></li></ul></li><li><span><a href="#Majority-vote-prediction-from-DBOW,PV-DMM-and-PV-DMC-predictions" data-toc-modified-id="Majority-vote-prediction-from-DBOW,PV-DMM-and-PV-DMC-predictions-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Majority vote prediction from DBOW,PV-DMM and PV-DMC predictions</a></span><ul class="toc-item"><li><span><a href="#Out-of-sample/-Never-before-seen-accuracy" data-toc-modified-id="Out-of-sample/-Never-before-seen-accuracy-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Out-of-sample/ Never before seen accuracy</a></span></li></ul></li><li><span><a href="#t-SNE-representation-of-learned-vectors" data-toc-modified-id="t-SNE-representation-of-learned-vectors-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>t-SNE representation of learned vectors</a></span></li><li><span><a href="#Checking-learned-keywords-for-each-bias-for-pictorial-representation" data-toc-modified-id="Checking-learned-keywords-for-each-bias-for-pictorial-representation-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Checking learned keywords for each bias for pictorial representation</a></span></li></ul></div>

In [560]:
import sqlite3
import pandas as pd
import requests
from lxml.html import fromstring
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.doc2vec import TaggedLineDocument,TaggedDocument
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import gensim.models as g
pd.options.mode.chained_assignment = None

/opt/anaconda3/envs/buan6341/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
[nltk_data] Downloading package stopwords to /Users/tapas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tapas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


- Separating data into df and out_of_sample_df 
- Creating new bias columns 

In [847]:
conn = sqlite3.connect('news_articles_all_keywords.sqlite')
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'",conn)
print(table)

                      name
0   news_abortion         
1   news_statue           
2   news_CHOP             
3   news_supreme_court    
4   news_vote_by_mail     
5   news_defund_the_police
6   news_mask             
7   news_obamacare        
8   news_bounty           
9   news_WHO              
10  news_gun_control      
11  news_climate_change   
12  news_hdxl             
13  news_dreamers         
14  news_china            
15  news_healthcare       
16  news_Affordable_Care  
17  news_bolton_book      


In [ ]:
df_news_abortion = pd.read_sql('select * from news_abortion', conn)
df_news_statue = pd.read_sql('select * from news_statue', conn)
df_news_CHOP = pd.read_sql('select * from news_CHOP', conn)
df_news_supreme_court = pd.read_sql('select * from news_supreme_court', conn)
df_news_vote_by_mail = pd.read_sql('select * from news_vote_by_mail', conn)
df_news_defund_the_police = pd.read_sql('select * from news_defund_the_police', conn)
df_news_mask = pd.read_sql('select * from news_mask', conn)
df_news_obamacare = pd.read_sql('select * from news_obamacare', conn)
df_news_bounty = pd.read_sql('select * from news_bounty', conn)
df_news_WHO = pd.read_sql('select * from news_WHO', conn)
df_news_gun_control = pd.read_sql('select * from news_gun_control', conn)
df_news_climate_change = pd.read_sql('select * from news_climate_change', conn)
df_news_dreamers = pd.read_sql('select * from news_dreamers', conn)
df_news_china = pd.read_sql('select * from news_china', conn)
df_news_healthcare = pd.read_sql('select * from news_healthcare', conn)
df_news_Affordable_Care = pd.read_sql('select * from news_Affordable_Care', conn)
df_news_news_bolton_book = pd.read_sql('select * from news_bolton_book', conn)

train_df_list = [df_news_abortion,df_news_statue,df_news_CHOP,df_news_supreme_court,df_news_vote_by_mail,
                 df_news_defund_the_police,df_news_mask,df_news_obamacare,df_news_bounty,df_news_WHO,
                df_news_gun_control,df_news_climate_change,df_news_dreamers,df_news_china,df_news_healthcare,
                df_news_Affordable_Care,df_news_news_bolton_book]

df = pd.DataFrame()
for temp_df in train_df_list:
    df = df.append(temp_df)

df = df[~(df['content_full'].isna())].reset_index(drop=True)
df['tag'] = df.index

df = pd.read_excel('/Users/tapas/Documents/Repos/news_content_v2.xlsx')
    


In [582]:
df = pd.read_excel('/Users/tapas/Documents/Repos/news_content_v3.xlsx')
out_of_sample_df = df[(df.topic == 'bolton_book') & (~df['content_full'].isnull())].reset_index(drop=True)
df = df[(df['select'] ==1) & (df.topic != 'bolton_book') & (~df['content_full'].isnull())].reset_index(drop=True)
df['bias'] = df['Alignment'].map({'left':1,'right':1,'center':0})
df['Alignment_int'] = df['Alignment'].map({'left':-1,'right':1,'center':0})
df['tag'] = df.index

In [859]:
df.topic.value_counts()

bounty               15
china                15
vote_by_mail         15
defund_the_police    15
dreamers             15
mask                 10
abortion             10
healthcare           9 
COVID                3 
WHO                  3 
gun_control          2 
climate_change       1 
sc                   1 
Name: topic, dtype: int64

In [829]:
stop_words = stop_words_3

In [620]:
## Tokenize Texts
stop_words_1 = ['washingtonpost','foxnews','usatoday','washington post','fox']
stop_words_2 = ['washingtonpost','foxnews','usatoday','washington post','fox','defund','police','vote','mail','mask',
              'obamacare','abortion','bounty','WHO','gun control','climate change','voting','china','healthcare',
               'dreamers','daca']

stop_words_3 = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves',
              'he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their',
              'theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are','was',
              'were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and',
              'but','if','or','because','as','until','while','of','at','by','for','with','about','against','between',
              'into','through','during','before','after','above','below','to','from','up','down','in','out','on','off'
              ,'over','under','again','further','then','once','here','there','when','where','why','how','all','any',
              'both','each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than',
              'too','very','s','t','can','will','just','don','should','now','said','could','would','washingtonpost','foxnews','usatoday',
               'washington post','fox']

stop_words_4 = ['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves',
              'he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their',
              'theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are','was',
              'were','be','been','being','have','has','had','having','do','does','did','doing','a','an','the','and',
              'but','if','or','because','as','until','while','of','at','by','for','with','about','against','between',
              'into','through','during','before','after','above','below','to','from','up','down','in','out','on','off'
              ,'over','under','again','further','then','once','here','there','when','where','why','how','all','any',
              'both','each','few','more','most','other','some','such','no','nor','not','only','own','same','so','than',
              'too','very','s','t','can','will','just','don','should','now','said','could','would','washingtonpost','foxnews','usatoday',
               'washington post','fox','defund','police','vote','mail','mask','obamacare','abortion','bounty','WHO','gun control','climate change',
               'voting','china','healthcare','dreamers','DACA']


In [130]:
negative_words = ['2-faced','2-faces','abnormal','abolish','abominable','abominably','abominate','abomination','abort','aborted','aborts','abrade','abrasive','abrupt','abruptly','abscond','absence','absent-minded','absentee','absurd','absurdity','absurdly','absurdness','abuse','abused','abuses','abusive','abysmal','abysmally','abyss','accidental','accost','accursed','accusation','accusations','accuse','accuses','accusing','accusingly','acerbate','acerbic','acerbically','ache','ached','aches','achey','aching','acrid','acridly','acridness','acrimonious','acrimoniously','acrimony','adamant','adamantly','addict','addicted','addicting','addicts','admonish','admonisher','admonishingly','admonishment','admonition','adulterate','adulterated','adulteration','adulterier','adversarial','adversary','adverse','adversity','afflict','affliction','afflictive','affront','afraid','aggravate','aggravating','aggravation','aggression','aggressive','aggressiveness','aggressor','aggrieve','aggrieved','aggrivation','aghast','agonies','agonize','agonizing','agonizingly','agony','aground','ail','ailing','ailment','aimless','alarm','alarmed','alarming','alarmingly','alienate','alienated','alienation','allegation','allegations','allege','allergic','allergies','allergy','aloof','altercation','ambiguity','ambiguous','ambivalence','ambivalent','ambush','amiss','amputate','anarchism','anarchist','anarchistic','anarchy','anemic','anger','angrily','angriness','angry','anguish','animosity','annihilate','annihilation','annoy','annoyance','annoyances','annoyed','annoying','annoyingly','annoys','anomalous','anomaly','antagonism','antagonist','antagonistic','antagonize','anti-','anti-american','anti-israeli','anti-occupation','anti-proliferation','anti-semites','anti-social','anti-us','anti-white','antipathy','antiquated','antithetical','anxieties','anxiety','anxious','anxiously','anxiousness','apathetic','apathetically','apathy','apocalypse','apocalyptic','apologist','apologists','appal','appall','appalled','appalling','appallingly','apprehension','apprehensions','apprehensive','apprehensively','arbitrary','arcane','archaic','arduous','arduously','argumentative','arrogance','arrogant','arrogantly','ashamed','asinine','asininely','asinininity','askance','asperse','aspersion','aspersions','assail','assassin','assassinate','assault','assult','astray','asunder','atrocious','atrocities','atrocity','atrophy','attack','attacks','audacious','audaciously','audaciousness','audacity','audiciously','austere','authoritarian','autocrat','autocratic','avalanche','avarice','avaricious','avariciously','avenge','averse','aversion','aweful','awful','awfully','awfulness','awkward','awkwardness','ax','babble','back-logged','back-wood','back-woods','backache','backaches','backaching','backbite','backbiting','backward','backwardness','backwood','backwoods','bad','badly','baffle','baffled','bafflement','baffling','bait','balk','banal','banalize','bane','banish','banishment','bankrupt','barbarian','barbaric','barbarically','barbarity','barbarous','barbarously','barren','baseless','bash','bashed','bashful','bashing','bastard','bastards','battered','battering','batty','bearish','beastly','bedlam','bedlamite','befoul','beg','beggar','beggarly','begging','beguile','belabor','belated','beleaguer','belie','belittle','belittled','belittling','bellicose','belligerence','belligerent','belligerently','bemoan','bemoaning','bemused','bent','berate','bereave','bereavement','bereft','berserk','beseech','beset','besiege','besmirch','bestial','betray','betrayal','betrayals','betrayer','betraying','betrays','bewail','beware','bewilder','bewildered','bewildering','bewilderingly','bewilderment','bewitch','bias','biased','biases','bicker','bickering','bid-rigging','bigotries','bigotry','bitch','bitchy','biting','bitingly','bitter','bitterly','bitterness','bizarre','blab','blabber','blackmail','blah','blame','blameworthy','bland','blandish','blaspheme','blasphemous','blasphemy','blasted','blatant','blatantly','blather','bleak','bleakly','bleakness','bleed','bleeding','bleeds','blemish','blind','blinding','blindingly','blindside','blister','blistering','bloated','blockage','blockhead','bloodshed','bloodthirsty','bloody','blotchy','blow','blunder','blundering','blunders','blunt','blur','bluring','blurred','blurring','blurry','blurs','blurt','boastful','boggle','bogus','boil','boiling','boisterous','bomb','bombard','bombardment','bombastic','bondage','bonkers','bore','bored','boredom','bores','boring','botch','bother','bothered','bothering','bothers','bothersome','bowdlerize','boycott','braggart','bragger','brainless','brainwash','brash','brashly','brashness','brat','bravado','brazen','brazenly','brazenness','breach','break','break-up','break-ups','breakdown','breaking','breaks','breakup','breakups','bribery','brimstone','bristle','brittle','broke','broken','broken-hearted','brood','browbeat','bruise','bruised','bruises','bruising','brusque','brutal','brutalising','brutalities','brutality','brutalize','brutalizing','brutally','brute','brutish','bs','buckle','bug','bugging','buggy','bugs','bulkier','bulkiness','bulky','bulkyness','bull****','bull----','bullies','bullshit','bullshyt','bully','bullying','bullyingly','bum','bump','bumped','bumping','bumpping','bumps','bumpy','bungle','bungler','bungling','bunk','burden','burdensome','burdensomely','burn','burned','burning','burns','bust','busts','busybody','butcher','butchery','buzzing','byzantine','cackle','calamities','calamitous','calamitously','calamity','callous','calumniate','calumniation','calumnies','calumnious','calumniously','calumny','cancer','cancerous','cannibal','cannibalize','capitulate','capricious','capriciously','capriciousness','capsize','careless','carelessness','caricature','carnage','carp','cartoonish','cash-strapped','castigate','castrated','casualty','cataclysm','cataclysmal','cataclysmic','cataclysmically','catastrophe','catastrophes','catastrophic','catastrophically','catastrophies','caustic','caustically','cautionary','cave','censure','chafe','chaff','chagrin','challenging','chaos','chaotic','chasten','chastise','chastisement','chatter','chatterbox','cheap','cheapen','cheaply','cheat','cheated','cheater','cheating','cheats','checkered','cheerless','cheesy','chide','childish','chill','chilly','chintzy','choke','choleric','choppy','chore','chronic','chunky','clamor','clamorous','clash','cliche','cliched','clique','clog','clogged','clogs','cloud','clouding','cloudy','clueless','clumsy','clunky','coarse','cocky','coerce','coercion','coercive','cold','coldly','collapse','collude','collusion','combative','combust','comical','commiserate','commonplace','commotion','commotions','complacent','complain','complained','complaining','complains','complaint','complaints','complex','complicated','complication','complicit','compulsion','compulsive','concede','conceded','conceit','conceited','concen','concens','concern','concerned','concerns','concession','concessions','condemn','condemnable','condemnation','condemned','condemns','condescend','condescending','condescendingly','condescension','confess','confession','confessions','confined','conflict','conflicted','conflicting','conflicts','confound','confounded','confounding','confront','confrontation','confrontational','confuse','confused','confuses','confusing','confusion','confusions','congested','congestion','cons','conscons','conservative','conspicuous','conspicuously','conspiracies','conspiracy','conspirator','conspiratorial','conspire','consternation','contagious','contaminate','contaminated','contaminates','contaminating','contamination','contempt','contemptible','contemptuous','contemptuously','contend','contention','contentious','contort','contortions','contradict','contradiction','contradictory','contrariness','contravene','contrive','contrived','controversial','controversy','convoluted','corrode','corrosion','corrosions','corrosive','corrupt','corrupted','corrupting','corruption','corrupts','corruptted','costlier','costly','counter-productive','counterproductive','coupists','covetous','coward','cowardly','crabby','crack','cracked','cracks','craftily','craftly','crafty','cramp','cramped','cramping','cranky','crap','crappy','craps','crash','crashed','crashes','crashing','crass','craven','cravenly','craze','crazily','craziness','crazy','creak','creaking','creaks','credulous','creep','creeping','creeps','creepy','crept','crime','criminal','cringe','cringed','cringes','cripple','crippled','cripples','crippling','crisis','critic','critical','criticism','criticisms','criticize','criticized','criticizing','critics','cronyism','crook','crooked','crooks','crowded','crowdedness','crude','cruel','crueler','cruelest','cruelly','cruelness','cruelties','cruelty','crumble','crumbling','crummy','crumple','crumpled','crumples','crush','crushed','crushing','cry','culpable','culprit','cumbersome','cunt','cunts','cuplrit','curse','cursed','curses','curt','cuss','cussed','cutthroat','cynical','cynicism','d*mn','damage','damaged','damages','damaging','damn','damnable','damnably','damnation','damned','damning','damper','danger','dangerous','dangerousness','dark','darken','darkened','darker','darkness','dastard','dastardly','daunt','daunting','dauntingly','dawdle','daze','dazed','dead','deadbeat','deadlock','deadly','deadweight','deaf','dearth','death','debacle','debase','debasement','debaser','debatable','debauch','debaucher','debauchery','debilitate','debilitating','debility','debt','debts','decadence','decadent','decay','decayed','deceit','deceitful','deceitfully','deceitfulness','deceive','deceiver','deceivers','deceiving','deception','deceptive','deceptively','declaim','decline','declines','declining','decrement','decrepit','decrepitude','decry','defamation','defamations','defamatory','defame','defect','defective','defects','defensive','defiance','defiant','defiantly','deficiencies','deficiency','deficient','defile','defiler','deform','deformed','defrauding','defunct','defy','degenerate','degenerately','degeneration','degradation','degrade','degrading','degradingly','dehumanization','dehumanize','deign','deject','dejected','dejectedly','dejection','delay','delayed','delaying','delays','delinquency','delinquent','delirious','delirium','delude','deluded','deluge','delusion','delusional','delusions','demean','demeaning','demise','demolish','demolisher','demon','demonic','demonize','demonized','demonizes','demonizing','demoralize','demoralizing','demoralizingly','denial','denied','denies','denigrate','denounce','dense','dent','dented','dents','denunciate','denunciation','denunciations','deny','denying','deplete','deplorable','deplorably','deplore','deploring','deploringly','deprave','depraved','depravedly','deprecate','depress','depressed','depressing','depressingly','depression','depressions','deprive','deprived','deride','derision','derisive','derisively','derisiveness','derogatory','desecrate','desert','desertion','desiccate','desiccated','desititute','desolate','desolately','desolation','despair','despairing','despairingly','desperate','desperately','desperation','despicable','despicably','despise','despised','despoil','despoiler','despondence','despondency','despondent','despondently','despot','despotic','despotism','destabilisation','destains','destitute','destitution','destroy','destroyer','destruction','destructive','desultory','deter','deteriorate','deteriorating','deterioration','deterrent','detest','detestable','detestably','detested','detesting','detests','detract','detracted','detracting','detraction','detracts','detriment','detrimental','devastate','devastated','devastates','devastating','devastatingly','devastation','deviate','deviation','devil','devilish','devilishly','devilment','devilry','devious','deviously','deviousness','devoid','diabolic','diabolical','diabolically','diametrically','diappointed','diatribe','diatribes','dick','dictator','dictatorial','die','die-hard','died','dies','difficult','difficulties','difficulty','diffidence','dilapidated','dilemma','dilly-dally','dim','dimmer','din','ding','dings','dinky','dire','direly','direness','dirt','dirtbag','dirtbags','dirts','dirty','disable','disabled','disaccord','disadvantage','disadvantaged','disadvantageous','disadvantages','disaffect','disaffected','disaffirm','disagree','disagreeable','disagreeably','disagreed','disagreeing','disagreement','disagrees','disallow','disapointed','disapointing','disapointment','disappoint','disappointed','disappointing','disappointingly','disappointment','disappointments','disappoints','disapprobation','disapproval','disapprove','disapproving','disarm','disarray','disaster','disasterous','disastrous','disastrously','disavow','disavowal','disbelief','disbelieve','disbeliever','disclaim','discombobulate','discomfit','discomfititure','discomfort','discompose','disconcert','disconcerted','disconcerting','disconcertingly','disconsolate','disconsolately','disconsolation','discontent','discontented','discontentedly','discontinued','discontinuity','discontinuous','discord','discordance','discordant','discountenance','discourage','discouragement','discouraging','discouragingly','discourteous','discourteously','discoutinous','discredit','discrepant','discriminate','discrimination','discriminatory','disdain','disdained','disdainful','disdainfully','disfavor','disgrace','disgraced','disgraceful','disgracefully','disgruntle','disgruntled','disgust','disgusted','disgustedly','disgustful','disgustfully','disgusting','disgustingly','dishearten','disheartening','dishearteningly','dishonest','dishonestly','dishonesty','dishonor','dishonorable','dishonorablely','disillusion','disillusioned','disillusionment','disillusions','disinclination','disinclined','disingenuous','disingenuously','disintegrate','disintegrated','disintegrates','disintegration','disinterest','disinterested','dislike','disliked','dislikes','disliking','dislocated','disloyal','disloyalty','dismal','dismally','dismalness','dismay','dismayed','dismaying','dismayingly','dismissive','dismissively','disobedience','disobedient','disobey','disoobedient','disorder','disordered','disorderly','disorganized','disorient','disoriented','disown','disparage','disparaging','disparagingly','dispensable','dispirit','dispirited','dispiritedly','dispiriting','displace','displaced','displease','displeased','displeasing','displeasure','disproportionate','disprove','disputable','dispute','disputed','disquiet','disquieting','disquietingly','disquietude','disregard','disregardful','disreputable','disrepute','disrespect','disrespectable','disrespectablity','disrespectful','disrespectfully','disrespectfulness','disrespecting','disrupt','disruption','disruptive','diss','dissapointed','dissappointed','dissappointing','dissatisfaction','dissatisfactory','dissatisfied','dissatisfies','dissatisfy','dissatisfying','dissed','dissemble','dissembler','dissension','dissent','dissenter','dissention','disservice','disses','dissidence','dissident','dissidents','dissing','dissocial','dissolute','dissolution','dissonance','dissonant','dissonantly','dissuade','dissuasive','distains','distaste','distasteful','distastefully','distort','distorted','distortion','distorts','distract','distracting','distraction','distraught','distraughtly','distraughtness','distress','distressed','distressing','distressingly','distrust','distrustful','distrusting','disturb','disturbance','disturbed','disturbing','disturbingly','disunity','disvalue','divergent','divisive','divisively','divisiveness','dizzing','dizzingly','dizzy','doddering','dodgey','dogged','doggedly','dogmatic','doldrums','domineer','domineering','donside','doom','doomed','doomsday','dope','doubt','doubtful','doubtfully','doubts','douchbag','douchebag','douchebags','downbeat','downcast','downer','downfall','downfallen','downgrade','downhearted','downheartedly','downhill','downside','downsides','downturn','downturns','drab','draconian','draconic','drag','dragged','dragging','dragoon','drags','drain','drained','draining','drains','drastic','drastically','drawback','drawbacks','dread','dreadful','dreadfully','dreadfulness','dreary','dripped','dripping','drippy','drips','drones','droop','droops','drop-out','drop-outs','dropout','dropouts','drought','drowning','drunk','drunkard','drunken','dubious','dubiously','dubitable','dud','dull','dullard','dumb','dumbfound','dump','dumped','dumping','dumps','dunce','dungeon','dungeons','dupe','dust','dusty','dwindling','dying','earsplitting','eccentric','eccentricity','effigy','effrontery','egocentric','egomania','egotism','egotistical','egotistically','egregious','egregiously','election-rigger','elimination','emaciated','emasculate','embarrass','embarrassing','embarrassingly','embarrassment','embattled','embroil','embroiled','embroilment','emergency','emphatic','emphatically','emptiness','encroach','encroachment','endanger','enemies','enemy','enervate','enfeeble','enflame','engulf','enjoin','enmity','enrage','enraged','enraging','enslave','entangle','entanglement','entrap','entrapment','envious','enviously','enviousness','epidemic','equivocal','erase','erode','erodes','erosion','err','errant','erratic','erratically','erroneous','erroneously','error','errors','eruptions','escapade','eschew','estranged','evade','evasion','evasive','evil','evildoer','evils','eviscerate','exacerbate','exagerate','exagerated','exagerates','exaggerate','exaggeration','exasperate','exasperated','exasperating','exasperatingly','exasperation','excessive','excessively','exclusion','excoriate','excruciating','excruciatingly','excuse','excuses','execrate','exhaust','exhausted','exhaustion','exhausts','exhorbitant','exhort','exile','exorbitant','exorbitantance','exorbitantly','expel','expensive','expire','expired','explode','exploit','exploitation','explosive','expropriate','expropriation','expulse','expunge','exterminate','extermination','extinguish','extort','extortion','extraneous','extravagance','extravagant','extravagantly','extremism','extremist','extremists','eyesore','f**k','fabricate','fabrication','facetious','facetiously','fail','failed','failing','fails','failure','failures','faint','fainthearted','faithless','fake','fall','fallacies','fallacious','fallaciously','fallaciousness','fallacy','fallen','falling','fallout','falls','false','falsehood','falsely','falsify','falter','faltered','famine','famished','fanatic','fanatical','fanatically','fanaticism','fanatics','fanciful','far-fetched','farce','farcical','farcical-yet-provocative','farcically','farfetched','fascism','fascist','fastidious','fastidiously','fastuous','fat','fat-cat','fat-cats','fatal','fatalistic','fatalistically','fatally','fatcat','fatcats','fateful','fatefully','fathomless','fatigue','fatigued','fatique','fatty','fatuity','fatuous','fatuously','fault','faults','faulty','fawningly','faze','fear','fearful','fearfully','fears','fearsome','feckless','feeble','feeblely','feebleminded','feign','feint','fell','felon','felonious','ferociously','ferocity','fetid','fever','feverish','fevers','fiasco','fib','fibber','fickle','fiction','fictional','fictitious','fidget','fidgety','fiend','fiendish','fierce','figurehead','filth','filthy','finagle','finicky','fissures','fist','flabbergast','flabbergasted','flagging','flagrant','flagrantly','flair','flairs','flak','flake','flakey','flakieness','flaking','flaky','flare','flares','flareup','flareups','flat-out','flaunt','flaw','flawed','flaws','flee','fleed','fleeing','fleer','flees','fleeting','flicering','flicker','flickering','flickers','flighty','flimflam','flimsy','flirt','flirty','floored','flounder','floundering','flout','fluster','foe','fool','fooled','foolhardy','foolish','foolishly','foolishness','forbid','forbidden','forbidding','forceful','foreboding','forebodingly','forfeit','forged','forgetful','forgetfully','forgetfulness','forlorn','forlornly','forsake','forsaken','forswear','foul','foully','foulness','fractious','fractiously','fracture','fragile','fragmented','frail','frantic','frantically','franticly','fraud','fraudulent','fraught','frazzle','frazzled','freak','freaking','freakish','freakishly','freaks','freeze','freezes','freezing','frenetic','frenetically','frenzied','frenzy','fret','fretful','frets','friction','frictions','fried','friggin','frigging','fright','frighten','frightening','frighteningly','frightful','frightfully','frigid','frost','frown','froze','frozen','fruitless','fruitlessly','frustrate','frustrated','frustrates','frustrating','frustratingly','frustration','frustrations','fuck','fucking','fudge','fugitive','full-blown','fulminate','fumble','fume','fumes','fundamentalism','funky','funnily','funny','furious','furiously','furor','fury','fuss','fussy','fustigate','fusty','futile','futilely','futility','fuzzy','gabble','gaff','gaffe','gainsay','gainsayer','gall','galling','gallingly','galls','gangster','gape','garbage','garish','gasp','gauche','gaudy','gawk','gawky','geezer','genocide','get-rich','ghastly','ghetto','ghosting','gibber','gibberish','gibe','giddy','gimmick','gimmicked','gimmicking','gimmicks','gimmicky','glare','glaringly','glib','glibly','glitch','glitches','gloatingly','gloom','gloomy','glower','glum','glut','gnawing','goad','goading','god-awful','goof','goofy','goon','gossip','graceless','gracelessly','graft','grainy','grapple','grate','grating','gravely','greasy','greed','greedy','grief','grievance','grievances','grieve','grieving','grievous','grievously','grim','grimace','grind','gripe','gripes','grisly','gritty','gross','grossly','grotesque','grouch','grouchy','groundless','grouse','growl','grudge','grudges','grudging','grudgingly','gruesome','gruesomely','gruff','grumble','grumpier','grumpiest','grumpily','grumpish','grumpy','guile','guilt','guiltily','guilty','gullible','gutless','gutter','hack','hacks','haggard','haggle','hairloss','halfhearted','halfheartedly','hallucinate','hallucination','hamper','hampered','handicapped','hang','hangs','haphazard','hapless','harangue','harass','harassed','harasses','harassment','harboring','harbors','hard','hard-hit','hard-line','hard-liner','hardball','harden','hardened','hardheaded','hardhearted','hardliner','hardliners','hardship','hardships','harm','harmed','harmful','harms','harpy','harridan','harried','harrow','harsh','harshly','hasseling','hassle','hassled','hassles','haste','hastily','hasty','hate','hated','hateful','hatefully','hatefulness','hater','haters','hates','hating','hatred','haughtily','haughty','haunt','haunting','havoc','hawkish','haywire','hazard','hazardous','haze','hazy','head-aches','headache','headaches','heartbreaker','heartbreaking','heartbreakingly','heartless','heathen','heavy-handed','heavyhearted','heck','heckle','heckled','heckles','hectic','hedge','hedonistic','heedless','hefty','hegemonism','hegemonistic','hegemony','heinous','hell','hell-bent','hellion','hells','helpless','helplessly','helplessness','heresy','heretic','heretical','hesitant','hestitant','hideous','hideously','hideousness','high-priced','hiliarious','hinder','hindrance','hiss','hissed','hissing','ho-hum','hoard','hoax','hobble','hogs','hollow','hoodium','hoodwink','hooligan','hopeless','hopelessly','hopelessness','horde','horrendous','horrendously','horrible','horrid','horrific','horrified','horrifies','horrify','horrifying','horrifys','hostage','hostile','hostilities','hostility','hotbeds','hothead','hotheaded','hothouse','hubris','huckster','hum','humid','humiliate','humiliating','humiliation','humming','hung','hurt','hurted','hurtful','hurting','hurts','hustler','hype','hypocricy','hypocrisy','hypocrite','hypocrites','hypocritical','hypocritically','hysteria','hysteric','hysterical','hysterically','hysterics','idiocies','idiocy','idiot','idiotic','idiotically','idiots','idle','ignoble','ignominious','ignominiously','ignominy','ignorance','ignorant','ignore','ill-advised','ill-conceived','ill-defined','ill-designed','ill-fated','ill-favored','ill-formed','ill-mannered','ill-natured','ill-sorted','ill-tempered','ill-treated','ill-treatment','ill-usage','ill-used','illegal','illegally','illegitimate','illicit','illiterate','illness','illogic','illogical','illogically','illusion','illusions','illusory','imaginary','imbalance','imbecile','imbroglio','immaterial','immature','imminence','imminently','immobilized','immoderate','immoderately','immodest','immoral','immorality','immorally','immovable','impair','impaired','impasse','impatience','impatient','impatiently','impeach','impedance','impede','impediment','impending','impenitent','imperfect','imperfection','imperfections','imperfectly','imperialist','imperil','imperious','imperiously','impermissible','impersonal','impertinent','impetuous','impetuously','impiety','impinge','impious','implacable','implausible','implausibly','implicate','implication','implode','impolite','impolitely','impolitic','importunate','importune','impose','imposers','imposing','imposition','impossible','impossiblity','impossibly','impotent','impoverish','impoverished','impractical','imprecate','imprecise','imprecisely','imprecision','imprison','imprisonment','improbability','improbable','improbably','improper','improperly','impropriety','imprudence','imprudent','impudence','impudent','impudently','impugn','impulsive','impulsively','impunity','impure','impurity','inability','inaccuracies','inaccuracy','inaccurate','inaccurately','inaction','inactive','inadequacy','inadequate','inadequately','inadverent','inadverently','inadvisable','inadvisably','inane','inanely','inappropriate','inappropriately','inapt','inaptitude','inarticulate','inattentive','inaudible','incapable','incapably','incautious','incendiary','incense','incessant','incessantly','incite','incitement','incivility','inclement','incognizant','incoherence','incoherent','incoherently','incommensurate','incomparable','incomparably','incompatability','incompatibility','incompatible','incompetence','incompetent','incompetently','incomplete','incompliant','incomprehensible','incomprehension','inconceivable','inconceivably','incongruous','incongruously','inconsequent','inconsequential','inconsequentially','inconsequently','inconsiderate','inconsiderately','inconsistence','inconsistencies','inconsistency','inconsistent','inconsolable','inconsolably','inconstant','inconvenience','inconveniently','incorrect','incorrectly','incorrigible','incorrigibly','incredulous','incredulously','inculcate','indecency','indecent','indecently','indecision','indecisive','indecisively','indecorum','indefensible','indelicate','indeterminable','indeterminably','indeterminate','indifference','indifferent','indigent','indignant','indignantly','indignation','indignity','indiscernible','indiscreet','indiscreetly','indiscretion','indiscriminate','indiscriminately','indiscriminating','indistinguishable','indoctrinate','indoctrination','indolent','indulge','ineffective','ineffectively','ineffectiveness','ineffectual','ineffectually','ineffectualness','inefficacious','inefficacy','inefficiency','inefficient','inefficiently','inelegance','inelegant','ineligible','ineloquent','ineloquently','inept','ineptitude','ineptly','inequalities','inequality','inequitable','inequitably','inequities','inescapable','inescapably','inessential','inevitable','inevitably','inexcusable','inexcusably','inexorable','inexorably','inexperience','inexperienced','inexpert','inexpertly','inexpiable','inexplainable','inextricable','inextricably','infamous','infamously','infamy','infected','infection','infections','inferior','inferiority','infernal','infest','infested','infidel','infidels','infiltrator','infiltrators','infirm','inflame','inflammation','inflammatory','inflammed','inflated','inflationary','inflexible','inflict','infraction','infringe','infringement','infringements','infuriate','infuriated','infuriating','infuriatingly','inglorious','ingrate','ingratitude','inhibit','inhibition','inhospitable','inhospitality','inhuman','inhumane','inhumanity','inimical','inimically','iniquitous','iniquity','injudicious','injure','injurious','injury','injustice','injustices','innuendo','inoperable','inopportune','inordinate','inordinately','insane','insanely','insanity','insatiable','insecure','insecurity','insensible','insensitive','insensitively','insensitivity','insidious','insidiously','insignificance','insignificant','insignificantly','insincere','insincerely','insincerity','insinuate','insinuating','insinuation','insociable','insolence','insolent','insolently','insolvent','insouciance','instability','instable','instigate','instigator','instigators','insubordinate','insubstantial','insubstantially','insufferable','insufferably','insufficiency','insufficient','insufficiently','insular','insult','insulted','insulting','insultingly','insults','insupportable','insupportably','insurmountable','insurmountably','insurrection','intefere','inteferes','intense','interfere','interference','interferes','intermittent','interrupt','interruption','interruptions','intimidate','intimidating','intimidatingly','intimidation','intolerable','intolerablely','intolerance','intoxicate','intractable','intransigence','intransigent','intrude','intrusion','intrusive','inundate','inundated','invader','invalid','invalidate','invalidity','invasive','invective','inveigle','invidious','invidiously','invidiousness','invisible','involuntarily','involuntary','irascible','irate','irately','ire','irk','irked','irking','irks','irksome','irksomely','irksomeness','irksomenesses','ironic','ironical','ironically','ironies','irony','irragularity','irrational','irrationalities','irrationality','irrationally','irrationals','irreconcilable','irrecoverable','irrecoverableness','irrecoverablenesses','irrecoverably','irredeemable','irredeemably','irreformable','irregular','irregularity','irrelevance','irrelevant','irreparable','irreplacible','irrepressible','irresolute','irresolvable','irresponsible','irresponsibly','irretating','irretrievable','irreversible','irritable','irritably','irritant','irritate','irritated','irritating','irritation','irritations','isolate','isolated','isolation','issue','issues','itch','itching','itchy','jabber','jaded','jagged','jam','jarring','jaundiced','jealous','jealously','jealousness','jealousy','jeer','jeering','jeeringly','jeers','jeopardize','jeopardy','jerk','jerky','jitter','jitters','jittery','job-killing','jobless','joke','joker','jolt','judder','juddering','judders','jumpy','junk','junky','junkyard','jutter','jutters','kaput','kill','killed','killer','killing','killjoy','kills','knave','knife','knock','knotted','kook','kooky','lack','lackadaisical','lacked','lackey','lackeys','lacking','lackluster','lacks','laconic','lag','lagged','lagging','laggy','lags','laid-off','lambast','lambaste','lame','lame-duck','lament','lamentable','lamentably','languid','languish','languor','languorous','languorously','lanky','lapse','lapsed','lapses','lascivious','last-ditch','latency','laughable','laughably','laughingstock','lawbreaker','lawbreaking','lawless','lawlessness','layoff','layoff-happy','lazy','leak','leakage','leakages','leaking','leaks','leaky','lech','lecher','lecherous','lechery','leech','leer','leery','left-leaning','lemon','lengthy','less-developed','lesser-known','letch','lethal','lethargic','lethargy','lewd','lewdly','lewdness','liability','liable','liar','liars','licentious','licentiously','licentiousness','lie','lied','lier','lies','life-threatening','lifeless','limit','limitation','limitations','limited','limits','limp','listless','litigious','little-known','livid','lividly','loath','loathe','loathing','loathly','loathsome','loathsomely','lone','loneliness','lonely','loner','lonesome','long-time','long-winded','longing','longingly','loophole','loopholes','loose','loot','lorn','lose','loser','losers','loses','losing','loss','losses','lost','loud','louder','lousy','loveless','lovelorn','low-rated','lowly','ludicrous','ludicrously','lugubrious','lukewarm','lull','lumpy','lunatic','lunaticism','lurch','lure','lurid','lurk','lurking','lying','macabre','mad','madden','maddening','maddeningly','madder','madly','madman','madness','maladjusted','maladjustment','malady','malaise','malcontent','malcontented','maledict','malevolence','malevolent','malevolently','malice','malicious','maliciously','maliciousness','malign','malignant','malodorous','maltreatment','mangle','mangled','mangles','mangling','mania','maniac','maniacal','manic','manipulate','manipulation','manipulative','manipulators','mar','marginal','marginally','martyrdom','martyrdom-seeking','mashed','massacre','massacres','matte','mawkish','mawkishly','mawkishness','meager','meaningless','meanness','measly','meddle','meddlesome','mediocre','mediocrity','melancholy','melodramatic','melodramatically','meltdown','menace','menacing','menacingly','mendacious','mendacity','menial','merciless','mercilessly','mess','messed','messes','messing','messy','midget','miff','militancy','mindless','mindlessly','mirage','mire','misalign','misaligned','misaligns','misapprehend','misbecome','misbecoming','misbegotten','misbehave','misbehavior','miscalculate','miscalculation','miscellaneous','mischief','mischievous','mischievously','misconception','misconceptions','miscreant','miscreants','misdirection','miser','miserable','miserableness','miserably','miseries','miserly','misery','misfit','misfortune','misgiving','misgivings','misguidance','misguide','misguided','mishandle','mishap','misinform','misinformed','misinterpret','misjudge','misjudgment','mislead','misleading','misleadingly','mislike','mismanage','mispronounce','mispronounced','mispronounces','misread','misreading','misrepresent','misrepresentation','miss','missed','misses','misstatement','mist','mistake','mistaken','mistakenly','mistakes','mistified','mistress','mistrust','mistrustful','mistrustfully','mists','misunderstand','misunderstanding','misunderstandings','misunderstood','misuse','moan','mobster','mock','mocked','mockeries','mockery','mocking','mockingly','mocks','molest','molestation','monotonous','monotony','monster','monstrosities','monstrosity','monstrous','monstrously','moody','moot','mope','morbid','morbidly','mordant','mordantly','moribund','moron','moronic','morons','mortification','mortified','mortify','mortifying','motionless','motley','mourn','mourner','mournful','mournfully','muddle','muddy','mudslinger','mudslinging','mulish','multi-polarization','mundane','murder','murderer','murderous','murderously','murky','muscle-flexing','mushy','musty','mysterious','mysteriously','mystery','mystify','myth','nag','nagging','naive','naively','narrower','nastily','nastiness','nasty','naughty','nauseate','nauseates','nauseating','nauseatingly','naÔve','nebulous','nebulously','needless','needlessly','needy','nefarious','nefariously','negate','negation','negative','negatives','negativity','neglect','neglected','negligence','negligent','nemesis','nepotism','nervous','nervously','nervousness','nettle','nettlesome','neurotic','neurotically','niggle','niggles','nightmare','nightmarish','nightmarishly','nitpick','nitpicking','noise','noises','noisier','noisy','non-confidence','nonexistent','nonresponsive','nonsense','nosey','notoriety','notorious','notoriously','noxious','nuisance','numb','obese','object','objection','objectionable','objections','oblique','obliterate','obliterated','oblivious','obnoxious','obnoxiously','obscene','obscenely','obscenity','obscure','obscured','obscures','obscurity','obsess','obsessive','obsessively','obsessiveness','obsolete','obstacle','obstinate','obstinately','obstruct','obstructed','obstructing','obstruction','obstructs','obtrusive','obtuse','occlude','occluded','occludes','occluding','odd','odder','oddest','oddities','oddity','oddly','odor','offence','offend','offender','offending','offenses','offensive','offensively','offensiveness','officious','ominous','ominously','omission','omit','one-sided','onerous','onerously','onslaught','opinionated','opponent','opportunistic','oppose','opposition','oppositions','oppress','oppression','oppressive','oppressively','oppressiveness','oppressors','ordeal','orphan','ostracize','outbreak','outburst','outbursts','outcast','outcry','outlaw','outmoded','outrage','outraged','outrageous','outrageously','outrageousness','outrages','outsider','over-acted','over-awe','over-balanced','over-hyped','over-priced','over-valuation','overact','overacted','overawe','overbalance','overbalanced','overbearing','overbearingly','overblown','overdo','overdone','overdue','overemphasize','overheat','overkill','overloaded','overlook','overpaid','overpayed','overplay','overpower','overpriced','overrated','overreach','overrun','overshadow','oversight','oversights','oversimplification','oversimplified','oversimplify','oversize','overstate','overstated','overstatement','overstatements','overstates','overtaxed','overthrow','overthrows','overturn','overweight','overwhelm','overwhelmed','overwhelming','overwhelmingly','overwhelms','overzealous','overzealously','overzelous','pain','painful','painfull','painfully','pains','pale','pales','paltry','pan','pandemonium','pander','pandering','panders','panic','panick','panicked','panicking','panicky','paradoxical','paradoxically','paralize','paralyzed','paranoia','paranoid','parasite','pariah','parody','partiality','partisan','partisans','passe','passive','passiveness','pathetic','pathetically','patronize','paucity','pauper','paupers','payback','peculiar','peculiarly','pedantic','peeled','peeve','peeved','peevish','peevishly','penalize','penalty','perfidious','perfidity','perfunctory','peril','perilous','perilously','perish','pernicious','perplex','perplexed','perplexing','perplexity','persecute','persecution','pertinacious','pertinaciously','pertinacity','perturb','perturbed','pervasive','perverse','perversely','perversion','perversity','pervert','perverted','perverts','pessimism','pessimistic','pessimistically','pest','pestilent','petrified','petrify','pettifog','petty','phobia','phobic','phony','picket','picketed','picketing','pickets','picky','pig','pigs','pillage','pillory','pimple','pinch','pique','pitiable','pitiful','pitifully','pitiless','pitilessly','pittance','pity','plagiarize','plague','plasticky','plaything','plea','pleas','plebeian','plight','plot','plotters','ploy','plunder','plunderer','pointless','pointlessly','poison','poisonous','poisonously','pokey','poky','polarisation','polemize','pollute','polluter','polluters','polution','pompous','poor','poorer','poorest','poorly','posturing','pout','poverty','powerless','prate','pratfall','prattle','precarious','precariously','precipitate','precipitous','predatory','predicament','prejudge','prejudice','prejudices','prejudicial','premeditated','preoccupy','preposterous','preposterously','presumptuous','presumptuously','pretence','pretend','pretense','pretentious','pretentiously','prevaricate','pricey','pricier','prick','prickle','prickles','prideful','prik','primitive','prison','prisoner','problem','problematic','problems','procrastinate','procrastinates','procrastination','profane','profanity','prohibit','prohibitive','prohibitively','propaganda','propagandize','proprietary','prosecute','protest','protested','protesting','protests','protracted','provocation','provocative','provoke','pry','pugnacious','pugnaciously','pugnacity','punch','punish','punishable','punitive','punk','puny','puppet','puppets','puzzled','puzzlement','puzzling','quack','qualm','qualms','quandary','quarrel','quarrellous','quarrellously','quarrels','quarrelsome','quash','queer','questionable','quibble','quibbles','quitter','rabid','racism','racist','racists','racy','radical','radicalization','radically','radicals','rage','ragged','raging','rail','raked','rampage','rampant','ramshackle','rancor','randomly','rankle','rant','ranted','ranting','rantingly','rants','rape','raped','raping','rascal','rascals','rash','rattle','rattled','rattles','ravage','raving','reactionary','rebellious','rebuff','rebuke','recalcitrant','recant','recession','recessionary','reckless','recklessly','recklessness','recoil','recourses','redundancy','redundant','refusal','refuse','refused','refuses','refusing','refutation','refute','refuted','refutes','refuting','regress','regression','regressive','regret','regreted','regretful','regretfully','regrets','regrettable','regrettably','regretted','reject','rejected','rejecting','rejection','rejects','relapse','relentless','relentlessly','relentlessness','reluctance','reluctant','reluctantly','remorse','remorseful','remorsefully','remorseless','remorselessly','remorselessness','renounce','renunciation','repel','repetitive','reprehensible','reprehensibly','reprehension','reprehensive','repress','repression','repressive','reprimand','reproach','reproachful','reprove','reprovingly','repudiate','repudiation','repugn','repugnance','repugnant','repugnantly','repulse','repulsed','repulsing','repulsive','repulsively','repulsiveness','resent','resentful','resentment','resignation','resigned','resistance','restless','restlessness','restrict','restricted','restriction','restrictive','resurgent','retaliate','retaliatory','retard','retarded','retardedness','retards','reticent','retract','retreat','retreated','revenge','revengeful','revengefully','revert','revile','reviled','revoke','revolt','revolting','revoltingly','revulsion','revulsive','rhapsodize','rhetoric','rhetorical','ricer','ridicule','ridicules','ridiculous','ridiculously','rife','rift','rifts','rigid','rigidity','rigidness','rile','riled','rip','rip-off','ripoff','ripped','risk','risks','risky','rival','rivalry','roadblocks','rocky','rogue','rollercoaster','rot','rotten','rough','rremediable','rubbish','rude','rue','ruffian','ruffle','ruin','ruined','ruining','ruinous','ruins','rumbling','rumor','rumors','rumours','rumple','run-down','runaway','rupture','rust','rusts','rusty','rut','ruthless','ruthlessly','ruthlessness','ruts','sabotage','sack','sacrificed','sad','sadden','sadly','sadness','sag','sagged','sagging','saggy','sags','salacious','sanctimonious','sap','sarcasm','sarcastic','sarcastically','sardonic','sardonically','sass','satirical','satirize','savage','savaged','savagery','savages','scaly','scam','scams','scandal','scandalize','scandalized','scandalous','scandalously','scandals','scandel','scandels','scant','scapegoat','scar','scarce','scarcely','scarcity','scare','scared','scarier','scariest','scarily','scarred','scars','scary','scathing','scathingly','sceptical','scoff','scoffingly','scold','scolded','scolding','scoldingly','scorching','scorchingly','scorn','scornful','scornfully','scoundrel','scourge','scowl','scramble','scrambled','scrambles','scrambling','scrap','scratch','scratched','scratches','scratchy','scream','screech','screw-up','screwed','screwed-up','screwy','scuff','scuffs','scum','scummy','second-class','second-tier','secretive','sedentary','seedy','seethe','seething','self-coup','self-criticism','self-defeating','self-destructive','self-humiliation','self-interest','self-interested','self-serving','selfinterested','selfish','selfishly','selfishness','semi-retarded','senile','sensationalize','senseless','senselessly','seriousness','sermonize','servitude','set-up','setback','setbacks','sever','severe','severity','sh*t','shabby','shadowy','shady','shake','shaky','shallow','sham','shambles','shame','shameful','shamefully','shamefulness','shameless','shamelessly','shamelessness','shark','sharply','shatter','shemale','shimmer','shimmy','shipwreck','shirk','shirker','shit','shiver','shock','shocked','shocking','shockingly','shoddy','short-lived','shortage','shortchange','shortcoming','shortcomings','shortness','shortsighted','shortsightedness','showdown','shrew','shriek','shrill','shrilly','shrivel','shroud','shrouded','shrug','shun','shunned','sick','sicken','sickening','sickeningly','sickly','sickness','sidetrack','sidetracked','siege','sillily','silly','simplistic','simplistically','sin','sinful','sinfully','sinister','sinisterly','sink','sinking','skeletons','skeptic','skeptical','skeptically','skepticism','sketchy','skimpy','skinny','skittish','skittishly','skulk','slack','slander','slanderer','slanderous','slanderously','slanders','slap','slashing','slaughter','slaughtered','slave','slaves','sleazy','slime','slog','slogged','slogging','slogs','sloooooooooooooow','sloooow','slooow','sloow','sloppily','sloppy','sloth','slothful','slow','slow-moving','slowed','slower','slowest','slowly','sloww','slowww','slowwww','slug','sluggish','slump','slumping','slumpping','slur','slut','sluts','sly','smack','smallish','smash','smear','smell','smelled','smelling','smells','smelly','smelt','smoke','smokescreen','smolder','smoldering','smother','smoulder','smouldering','smudge','smudged','smudges','smudging','smug','smugly','smut','smuttier','smuttiest','smutty','snag','snagged','snagging','snags','snappish','snappishly','snare','snarky','snarl','sneak','sneakily','sneaky','sneer','sneering','sneeringly','snob','snobbish','snobby','snobish','snobs','snub','so-cal','soapy','sob','sober','sobering','solemn','solicitude','somber','sore','sorely','soreness','sorrow','sorrowful','sorrowfully','sorry','sour','sourly','spade','spank','spendy','spew','spewed','spewing','spews','spilling','spinster','spiritless','spite','spiteful','spitefully','spitefulness','splatter','split','splitting','spoil','spoilage','spoilages','spoiled','spoilled','spoils','spook','spookier','spookiest','spookily','spooky','spoon-fed','spoon-feed','spoonfed','sporadic','spotty','spurious','spurn','sputter','squabble','squabbling','squander','squash','squeak','squeaks','squeaky','squeal','squealing','squeals','squirm','stab','stagnant','stagnate','stagnation','staid','stain','stains','stale','stalemate','stall','stalls','stammer','stampede','standstill','stark','starkly','startle','startling','startlingly','starvation','starve','static','steal','stealing','steals','steep','steeply','stench','stereotype','stereotypical','stereotypically','stern','stew','sticky','stiff','stiffness','stifle','stifling','stiflingly','stigma','stigmatize','sting','stinging','stingingly','stingy','stink','stinks','stodgy','stole','stolen','stooge','stooges','stormy','straggle','straggler','strain','strained','straining','strange','strangely','stranger','strangest','strangle','streaky','strenuous','stress','stresses','stressful','stressfully','stricken','strict','strictly','strident','stridently','strife','strike','stringent','stringently','struck','struggle','struggled','struggles','struggling','strut','stubborn','stubbornly','stubbornness','stuck','stuffy','stumble','stumbled','stumbles','stump','stumped','stumps','stun','stunt','stunted','stupid','stupidest','stupidity','stupidly','stupified','stupify','stupor','stutter','stuttered','stuttering','stutters','sty','stymied','sub-par','subdued','subjected','subjection','subjugate','subjugation','submissive','subordinate','subpoena','subpoenas','subservience','subservient','substandard','subtract','subversion','subversive','subversively','subvert','succumb','suck','sucked','sucker','sucks','sucky','sue','sued','sueing','sues','suffer','suffered','sufferer','sufferers','suffering','suffers','suffocate','sugar-coat','sugar-coated','sugarcoated','suicidal','suicide','sulk','sullen','sully','sunder','sunk','sunken','superficial','superficiality','superficially','superfluous','superstition','superstitious','suppress','suppression','surrender','susceptible','suspect','suspicion','suspicions','suspicious','suspiciously','swagger','swamped','sweaty','swelled','swelling','swindle','swipe','swollen','symptom','symptoms','syndrome','taboo','tacky','taint','tainted','tamper','tangle','tangled','tangles','tank','tanked','tanks','tantrum','tardy','tarnish','tarnished','tarnishes','tarnishing','tattered','taunt','taunting','tauntingly','taunts','taut','tawdry','taxing','tease','teasingly','tedious','tediously','temerity','temper','tempest','temptation','tenderness','tense','tension','tentative','tentatively','tenuous','tenuously','tepid','terrible','terribleness','terribly','terror','terror-genic','terrorism','terrorize','testily','testy','tetchily','tetchy','thankless','thicker','thirst','thorny','thoughtless','thoughtlessly','thoughtlessness','thrash','threat','threaten','threatening','threats','threesome','throb','throbbed','throbbing','throbs','throttle','thug','thumb-down','thumbs-down','thwart','time-consuming','timid','timidity','timidly','timidness','tin-y','tingled','tingling','tired','tiresome','tiring','tiringly','toil','toll','top-heavy','topple','torment','tormented','torrent','tortuous','torture','tortured','tortures','torturing','torturous','torturously','totalitarian','touchy','toughness','tout','touted','touts','toxic','traduce','tragedy','tragic','tragically','traitor','traitorous','traitorously','tramp','trample','transgress','transgression','trap','traped','trapped','trash','trashed','trashy','trauma','traumatic','traumatically','traumatize','traumatized','travesties','travesty','treacherous','treacherously','treachery','treason','treasonous','trick','tricked','trickery','tricky','trivial','trivialize','trouble','troubled','troublemaker','troubles','troublesome','troublesomely','troubling','troublingly','truant','tumble','tumbled','tumbles','tumultuous','turbulent','turmoil','twist','twisted','twists','two-faced','two-faces','tyrannical','tyrannically','tyranny','tyrant','ugh','uglier','ugliest','ugliness','ugly','ulterior','ultimatum','ultimatums','ultra-hardline','un-viewable','unable','unacceptable','unacceptablely','unacceptably','unaccessible','unaccustomed','unachievable','unaffordable','unappealing','unattractive','unauthentic','unavailable','unavoidably','unbearable','unbearablely','unbelievable','unbelievably','uncaring','uncertain','uncivil','uncivilized','unclean','unclear','uncollectible','uncomfortable','uncomfortably','uncomfy','uncompetitive','uncompromising','uncompromisingly','unconfirmed','unconstitutional','uncontrolled','unconvincing','unconvincingly','uncooperative','uncouth','uncreative','undecided','undefined','undependability','undependable','undercut','undercuts','undercutting','underdog','underestimate','underlings','undermine','undermined','undermines','undermining','underpaid','underpowered','undersized','undesirable','undetermined','undid','undignified','undissolved','undocumented','undone','undue','unease','uneasily','uneasiness','uneasy','uneconomical','unemployed','unequal','unethical','uneven','uneventful','unexpected','unexpectedly','unexplained','unfairly','unfaithful','unfaithfully','unfamiliar','unfavorable','unfeeling','unfinished','unfit','unforeseen','unforgiving','unfortunate','unfortunately','unfounded','unfriendly','unfulfilled','unfunded','ungovernable','ungrateful','unhappily','unhappiness','unhappy','unhealthy','unhelpful','unilateralism','unimaginable','unimaginably','unimportant','uninformed','uninsured','unintelligible','unintelligile','unipolar','unjust','unjustifiable','unjustifiably','unjustified','unjustly','unkind','unkindly','unknown','unlamentable','unlamentably','unlawful','unlawfully','unlawfulness','unleash','unlicensed','unlikely','unlucky','unmoved','unnatural','unnaturally','unnecessary','unneeded','unnerve','unnerved','unnerving','unnervingly','unnoticed','unobserved','unorthodox','unorthodoxy','unpleasant','unpleasantries','unpopular','unpredictable','unprepared','unproductive','unprofitable','unprove','unproved','unproven','unproves','unproving','unqualified','unravel','unraveled','unreachable','unreadable','unrealistic','unreasonable','unreasonably','unrelenting','unrelentingly','unreliability','unreliable','unresolved','unresponsive','unrest','unruly','unsafe','unsatisfactory','unsavory','unscrupulous','unscrupulously','unsecure','unseemly','unsettle','unsettled','unsettling','unsettlingly','unskilled','unsophisticated','unsound','unspeakable','unspeakablely','unspecified','unstable','unsteadily','unsteadiness','unsteady','unsuccessful','unsuccessfully','unsupported','unsupportive','unsure','unsuspecting','unsustainable','untenable','untested','unthinkable','unthinkably','untimely','untouched','untrue','untrustworthy','untruthful','unusable','unusably','unuseable','unuseably','unusual','unusually','unviewable','unwanted','unwarranted','unwatchable','unwelcome','unwell','unwieldy','unwilling','unwillingly','unwillingness','unwise','unwisely','unworkable','unworthy','unyielding','upbraid','upheaval','uprising','uproar','uproarious','uproariously','uproarous','uproarously','uproot','upset','upseting','upsets','upsetting','upsettingly','urgent','useless','usurp','usurper','utterly','vagrant','vague','vagueness','vain','vainly','vanity','vehement','vehemently','vengeance','vengeful','vengefully','vengefulness','venom','venomous','venomously','vent','vestiges','vex','vexation','vexing','vexingly','vibrate','vibrated','vibrates','vibrating','vibration','vice','vicious','viciously','viciousness','victimize','vile','vileness','vilify','villainous','villainously','villains','villian','villianous','villianously','villify','vindictive','vindictively','vindictiveness','violate','violation','violator','violators','violent','violently','viper','virulence','virulent','virulently','virus','vociferous','vociferously','volatile','volatility','vomit','vomited','vomiting','vomits','vulgar','vulnerable','wack','wail','wallow','wane','waning','wanton','war-like','warily','wariness','warlike','warned','warning','warp','warped','wary','washed-out','waste','wasted','wasteful','wastefulness','wasting','water-down','watered-down','wayward','weak','weaken','weakening','weaker','weakness','weaknesses','weariness','wearisome','weary','wedge','weed','weep','weird','weirdly','wheedle','whimper','whine','whining','whiny','whips','whore','whores','wicked','wickedly','wickedness','wild','wildly','wiles','wilt','wily','wimpy','wince','wobble','wobbled','wobbles','woe','woebegone','woeful','woefully','womanizer','womanizing','worn','worried','worriedly','worrier','worries','worrisome','worry','worrying','worryingly','worse','worsen','worsening','worst','worthless','worthlessly','worthlessness','wound','wounds','wrangle','wrath','wreak','wreaked','wreaks','wreck','wrest','wrestle','wretch','wretched','wretchedly','wretchedness','wrinkle','wrinkled','wrinkles','wrip','wripped','wripping','writhe','wrong','wrongful','wrongly','wrought','yawn','zap','zapped','zaps','zealot','zealous','zealously','zombie']
negative_words_set = set(negative_words)


In [478]:
## Clean and tokenize texts
import re

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

def remove_stop_words(content):
    return [word for word in content if word.lower() not in stop_words]

stemmer = PorterStemmer()

def stem_words(text):
    try:
        #text = [stemmer.stem(word) for word in text]
        text = [word for word in text if len(word) > 1]
    except IndexError:
        pass
    return text

def clean_text(text):
    text = re.sub("[^a-zA-Z]"," ",text)
    text = text.lower()
    return text


def apply_all(text):
    return remove_stop_words(tokenize_text(clean_text(text)))

In [479]:
## Clean and tokenize and keep onky negative words sentences in texts
import re
def sent_has_negwords(sent):
    input_set = set(nltk.word_tokenize(sent))
    if len(list(negative_words_set & input_set)) > 0:
        return True
    else:
        return False
        


def keep_negative_sentences(text):
    sent_str = []
    for sent in text.split('.'):
        if sent_has_negwords(sent):
            #sent = list(nltk.word_tokenize(sent))
            #sent = re.sub("[^a-zA-Z]"," ",sent)
            sent = remove_stop_words(tokenize_text(clean_text(text)))
            sent_str+=sent
    #print(sent_str)
    #print('**************************************************************************************')
    return sent_str
            
    

- creating different content columns 
- all texts, filtered texts (negative keyword sentences) , filtered texts 2 (negative keyword sentences of capped length)
- some washingtonpost articles were too long compared to oters, so articles are capped to a certain length

In [623]:
df['content_filtered'] = df.apply(lambda x: keep_negative_sentences(x['content_full']),axis=1)
out_of_sample_df['content_filtered'] = out_of_sample_df.apply(lambda x: keep_negative_sentences(x['content_full']),axis=1)
df['content_filtered_2'] = df.apply(lambda x: x['content_filtered'][0:20000],axis=1)
out_of_sample_df['content_filtered_2'] = out_of_sample_df.apply(lambda x: x['content_filtered'][0:20000],axis=1)
#df['tokenized_content'] = df.apply(lambda x : apply_all(x['content_full']),axis=1)
#df['tokenized_title'] = df.apply(lambda x : apply_all(x['title']),axis=1)

In [624]:
train_df,test_df = train_test_split(df,test_size=0.3)

In [483]:
from gensim.models import LdaModel
from gensim import models, corpora, similarities

num_topics = 5
dictionary = corpora.Dictionary(df['content_filtered_2'])
corpus = [dictionary.doc2bow(doc) for doc in df['content_filtered_2']]

## Topic Modelling using LDA

In [830]:
lda = LdaModel(corpus=corpus,num_topics=num_topics,id2word=dictionary,alpha=0.5e-2,eta=0.5e-2,minimum_probability=0.0,passes=50)

In [831]:
lda.show_topics(num_topics=5,num_words=20)

[(0,
  '0.014*"police" + 0.012*"court" + 0.010*"law" + 0.009*"abortion" + 0.006*"people" + 0.005*"masks" + 0.004*"state" + 0.004*"black" + 0.004*"louisiana" + 0.004*"case" + 0.004*"mask" + 0.004*"justice" + 0.004*"trump" + 0.003*"one" + 0.003*"supreme" + 0.003*"public" + 0.003*"health" + 0.003*"officers" + 0.003*"floyd" + 0.003*"even"'),
 (1,
  '0.017*"mail" + 0.011*"voting" + 0.011*"trump" + 0.011*"election" + 0.011*"ballots" + 0.010*"health" + 0.009*"states" + 0.009*"care" + 0.008*"vote" + 0.007*"voters" + 0.006*"democrats" + 0.006*"ballot" + 0.006*"pandemic" + 0.006*"court" + 0.006*"aca" + 0.006*"law" + 0.005*"republicans" + 0.005*"fraud" + 0.005*"president" + 0.005*"people"'),
 (2,
  '0.032*"trump" + 0.016*"president" + 0.015*"intelligence" + 0.010*"russia" + 0.008*"bounties" + 0.007*"house" + 0.007*"russian" + 0.006*"white" + 0.006*"reports" + 0.005*"national" + 0.005*"briefed" + 0.005*"bolton" + 0.005*"news" + 0.005*"afghanistan" + 0.005*"taliban" + 0.005*"troops" + 0.005*"securi

In [845]:
lda.show_topic(topicid=1,topn=20)

[('mail', 0.017362358),
 ('voting', 0.011491839),
 ('trump', 0.010871863),
 ('election', 0.010712173),
 ('ballots', 0.010520697),
 ('health', 0.009834845),
 ('states', 0.009461433),
 ('care', 0.008507006),
 ('vote', 0.008374221),
 ('voters', 0.007079174),
 ('democrats', 0.006315177),
 ('ballot', 0.006076458),
 ('pandemic', 0.005842355),
 ('court', 0.0058362596),
 ('aca', 0.0057932884),
 ('law', 0.0055250647),
 ('republicans', 0.005484115),
 ('fraud', 0.0054074824),
 ('president', 0.0052694473),
 ('people', 0.0051296833)]

In [846]:
lda.show_topic(topicid=0,topn=20)

[('police', 0.014490699),
 ('court', 0.01165123),
 ('law', 0.010262634),
 ('abortion', 0.008874021),
 ('people', 0.005819376),
 ('masks', 0.0046457527),
 ('state', 0.00439744),
 ('black', 0.0043186713),
 ('louisiana', 0.0041384664),
 ('case', 0.0037109582),
 ('mask', 0.0036647636),
 ('justice', 0.0036644598),
 ('trump', 0.0035463243),
 ('one', 0.0034773415),
 ('supreme', 0.0034769257),
 ('public', 0.0033843624),
 ('health', 0.003234547),
 ('officers', 0.0031742407),
 ('floyd', 0.002870055),
 ('even', 0.002853896)]

- Define tranform for converting document into vectors

In [592]:
import multiprocessing as mp
from sklearn import utils
import numpy as np
from gensim.models.callbacks import CallbackAny2Vec


class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        #loss = model.get_latest_training_loss()
        self.epoch += 1
        print('Epoch Number : {}'.format(self.epoch))
        print(model.docvecs.doctag_syn0)
        
class Doc2VecTransformer(BaseEstimator):
    def __init__(self,vector_size=300,window_size=15,min_count=1,dm=0,
                 dm_mean=0,dm_concat=0,epochs=100,sample=1e-5,model_path=''):
        self.vector_size=vector_size
        self.window_size=window_size
        self.min_count=min_count
        self.workers=mp.cpu_count()
        self.dm=dm
        self.epochs=epochs
        self.sample = sample
        self.dm_mean = dm_mean
        self.dm_concat = dm_concat
        #self.learning_rate = learning_rate
        self._model = None
        self.model_path = model_path
        #self.pretrained_emb=pretrained_emb, 
    def fit(self,X,Y=None):
        #tagged_x = [TaggedDocument(apply_all(row['content_full']),[index]) for index,row in X.iterrows()]
        tagged_x = [TaggedDocument(row['content_filtered_2'],[index]) for index,row in X.iterrows()]
        #tagged_x_array = iter(utils.shuffle([x for x in tagged_x]))
        #print(tagged_x_array.__next__())
        #print('*******************************************************************')
        #print(tagged_x_array.__next__())
        #train_tagged = X.apply(lambda x : TaggedDocument(words=x.tokenized_content,tags=[x.tag]),axis=1)
        model = g.Doc2Vec(vector_size=self.vector_size, window=self.window_size, min_count=self.min_count,
                          sample=self.sample, workers=self.workers, hs=0, dm=self.dm,negative=5, dbow_words=1,
                         dm_mean = self.dm_mean,dm_concat=self.dm_concat)
        
        model.build_vocab([x for x in tqdm(tagged_x)])
        model.train(utils.shuffle([x for x in tqdm(tagged_x)]), total_examples=len(tagged_x), epochs=self.epochs)
        self._model = model
        model.save(self.model_path)
        return self

    def transform(self,X):
        #return np.asmatrix(np.array([self._model.infer_vector(apply_all(row['content_full'])) for index,row in X.iterrows()])) 
        return np.asmatrix(np.array([self._model.infer_vector(row['content_filtered_2']) for index,row in X.iterrows()])) 

    
    

- Define Pipeline and GridSearch for Document embedding and classification using Logistic Regression

In [772]:
from sklearn.model_selection import GridSearchCV
from  xgboost import XGBClassifier
from sklearn.metrics import f1_score,fbeta_score,make_scorer


f2_scorer = make_scorer(fbeta_score, beta=0.5,average='weighted')
news_pipeline = Pipeline(steps=[('doc2vec',Doc2VecTransformer()),
                                ('logistic', LogisticRegression(max_iter=300))])



#news_pipeline = Pipeline(steps=[('doc2vec',Doc2VecTransformer()),('xgb', XGBClassifier())])

f2_scorer_2 = make_scorer(fbeta_score, beta=2.0,average='weighted')
f1_scorer = make_scorer(f1_score,average='weighted')

def run_grid_search(dm=0,dm_concat=0,dm_mean=0,model_path='',stop_words=[]):
    param_grid = {
        'doc2vec__vector_size': [100],
        'doc2vec__window_size': [10,20],
        'doc2vec__min_count':[1,2],
        'doc2vec__sample':[1e-5],
        'doc2vec__dm':[dm],
        'doc2vec__dm_concat':[dm_concat],
        'doc2vec__dm_mean':[dm_mean],
        'doc2vec__model_path':[model_path],
        'doc2vec__epochs':[50]
        }
    '''
        'xgb__max_depth' : [4,6],
        'xgb__n_estimators' : [100],
        'xgb__min_child_weight' : [3,5]
    '''

        
    stop_words = stop_words
    news_grid = GridSearchCV(news_pipeline, param_grid=param_grid, cv=5, n_jobs=-1,scoring=f2_scorer)
    news_grid.fit(train_df,train_df[['Alignment']].values.ravel())
    predictions_y = news_grid.predict(test_df)
    return news_grid,predictions_y


def run_grid_search_2(dm=0,dm_concat=0,dm_mean=0,model_path='',stop_words=[]):
    param_grid = {
        'doc2vec__vector_size': [100],
        'doc2vec__window_size': [10,20],
        'doc2vec__min_count':[1,2],
        'doc2vec__sample':[1e-5],
        'doc2vec__dm':[dm],
        'doc2vec__dm_concat':[dm_concat],
        'doc2vec__dm_mean':[dm_mean],
        'doc2vec__model_path':[model_path],
        'doc2vec__epochs':[50]
        }
    '''
        'xgb__max_depth' : [4,6],
        'xgb__n_estimators' : [100],
        'xgb__min_child_weight' : [3,5]
    '''

        
    stop_words = stop_words
    news_grid = GridSearchCV(news_pipeline, param_grid=param_grid, cv=5, n_jobs=-1,scoring=f2_scorer_2)
    news_grid.fit(train_df,train_df[['Alignment']].values.ravel())
    predictions_y = news_grid.predict(test_df)
    return news_grid,predictions_y


## DBOW MODEL

In [687]:
#beta =0.5
path = '/Users/tapas/Documents/Repos/news_analysis/model_bow.bin'
news_grid,predictions_y = run_grid_search(model_path=path,dm=0,dm_mean=0,dm_concat=0,stop_words=stop_words_4)

print("Best Cross Validation Score:",news_grid.best_score_)
print("Best Params",news_grid.best_params_)
print('F-Beta Score: ', news_grid.score(test_df[['content_filtered_2']], predictions_y))

prediction_df = test_df
prediction_df['predictions_bow'] = predictions_y
prediction_df[['title','Alignment','predictions_bow']]

predictions_y2 = news_grid.predict(out_of_sample_df)
out_of_sample_df['predictions_bow'] = predictions_y2
'''
Best Cross Validation Score: 0.34718448930776735
Best Params {'doc2vec__dm': 0, 'doc2vec__dm_concat': 0, 'doc2vec__dm_mean': 0, 'doc2vec__epochs': 50, 'doc2vec__min_count': 2, 'doc2vec__model_path': '/Users/tapas/Documents/Repos/news_analysis/model_bow.bin', 'doc2vec__sample': 1e-05, 'doc2vec__vector_size': 100, 'doc2vec__window_size': 15}
F-Beta Score:  0.9778126635269493
'''

100%|██████████| 79/79 [00:00<00:00, 32110.67it/s]


Best Cross Validation Score: 0.3627253905514775
Best Params {'doc2vec__dm': 0, 'doc2vec__dm_concat': 0, 'doc2vec__dm_mean': 0, 'doc2vec__epochs': 50, 'doc2vec__min_count': 1, 'doc2vec__model_path': '/Users/tapas/Documents/Repos/news_analysis/model_bow.bin', 'doc2vec__sample': 1e-05, 'doc2vec__vector_size': 100, 'doc2vec__window_size': 15}
F-Beta Score:  0.9844155844155844


"\nBest Cross Validation Score: 0.34718448930776735\nBest Params {'doc2vec__dm': 0, 'doc2vec__dm_concat': 0, 'doc2vec__dm_mean': 0, 'doc2vec__epochs': 50, 'doc2vec__min_count': 2, 'doc2vec__model_path': '/Users/tapas/Documents/Repos/news_analysis/model_bow.bin', 'doc2vec__sample': 1e-05, 'doc2vec__vector_size': 100, 'doc2vec__window_size': 15}\nF-Beta Score:  0.9778126635269493\n"

In [688]:
pd.set_option('display.max_colwidth',-1)
prediction_df[['title','Alignment','predictions_bow','tag']]

,title,Alignment,predictions_bow,tag
26,Defund police in schools? How the movement got momentum after George Floyd's death,center,center,26
2,"With abortion ruling, Roberts reasserts his role and Supreme Court’s independence",left,center,2
24,"Trump, at Fox News town hall, says mail-in voting is 'biggest risk' to fair November election, says Dems are 'destroying our country'",right,center,24
35,"Clyburn says 'nobody is going to defund the police,' calls for 'reimagining' of law enforcement",center,center,35
90,Steve Levy: DACA decision – Supreme Court's blow to common sense and the rule of law,right,center,90
6,"Supreme Court strikes down abortion clinic restrictions in Louisiana, a defeat for conservatives",left,center,6
22,Liz Harrington: Democrats seek mail-in voting so they can cheat their way to victory in November,right,center,22
62,National security adviser says Trump’s CIA briefer decided not to share Russia bounty intel,right,left,62
99,Joe Biden needs a China narrative that refutes today’s political angst,left,center,99
50,People furious over Detroit Zoo’s outdoor mask policy: 'See how many people pass out' in warmer weather,right,center,50


### Checking similarity vectors in DBOW Model

In [756]:
model_bow = g.Doc2Vec.load("/Users/tapas/Documents/Repos/news_analysis/model_bow.bin")

def get_paragraph_vector(content):
    vec = model_bow.infer_vector(content)
    return vec

test_df['paragraph_vector'] = test_df.apply(lambda x: get_paragraph_vector(x['content_filtered_2']),axis=1)
print(test_df[test_df.tag == 63]['title'])
test_vector = test_df[test_df.tag == 63]['paragraph_vector']
sims_posititve = model_bow.docvecs.most_similar(test_vector)
sims_positive



63    Trump’s ugly new spin on Russia bounties actually incriminates him
Name: title, dtype: object


[(101, 0.4486703872680664),
 (83, 0.3195067346096039),
 (85, 0.29169732332229614),
 (84, 0.2863686978816986),
 (93, 0.2789568603038788),
 (45, 0.25913944840431213),
 (89, 0.23757733404636383),
 (53, 0.23568525910377502),
 (109, 0.23428824543952942),
 (82, 0.21257035434246063)]

- Positive similarity

In [759]:
train_df[train_df.tag == 101][['title','Alignment']]

,title,Alignment
101,Russia. Ukraine. China. Bolton account highlights pattern of Trump welcoming foreign political help.,left


In [761]:
train_df[train_df.tag == 83][['title','Alignment']]

,title,Alignment
83,'We won': DACA recipients overwhelmed by surprise Supreme Court victory over Trump,left


In [762]:
train_df[train_df.tag == 85][['title','Alignment']]

,title,Alignment
85,"Trump blasts DACA decision, asking if people get the impression 'the Supreme Court doesn't like me?'",left


In [863]:
train_df[train_df.tag == 7][['title','Alignment']]

,title,Alignment
7,Supreme Court strikes down Louisiana law on abortion clinic restrictions,center


- Negative Similarity

In [749]:
sims_negative = model_bow.docvecs.most_similar(negative=test_vector)
sims_negative

[(35, 0.17234018445014954),
 (103, 0.1506379246711731),
 (79, 0.1496298909187317),
 (29, 0.14263445138931274),
 (99, 0.1422889083623886),
 (102, 0.12319527566432953),
 (55, 0.11329978704452515),
 (13, 0.08330686390399933),
 (33, 0.08016876876354218),
 (62, 0.07218089699745178)]

In [696]:
test_df[test_df.tag == 35][['title','Alignment']]

,title,Alignment
35,"Clyburn says 'nobody is going to defund the police,' calls for 'reimagining' of law enforcement",center


In [697]:
test_df[test_df.tag == 103][['title','Alignment']]

,title,Alignment
103,"US, China ramp up South China Sea tension with new military drills",center


In [698]:
test_df[test_df.tag == 79][['title','Alignment']]

,title,Alignment
79,Rep. Greg Walden: Green New Deal would cripple economy – Republicans have better way to fight climate change,right


In [753]:
print(test_df[test_df.tag == 94]['title'])
test_vector = test_df[test_df.tag == 94]['paragraph_vector']
sims_positive = model_bow.docvecs.most_similar(positive=test_vector)
sims_positive

94    Bolton book and Supreme Court ruling on ‘dreamers’ amount to twin defeats to Trump
Name: title, dtype: object


[(101, 0.4486703872680664),
 (83, 0.3195067346096039),
 (85, 0.29169732332229614),
 (84, 0.2863686978816986),
 (93, 0.2789568603038788),
 (45, 0.25913944840431213),
 (89, 0.23757733404636383),
 (53, 0.23568525910377502),
 (109, 0.23428824543952942),
 (82, 0.21257035434246063)]

In [677]:
train_df[train_df.tag == 101][['title','Alignment']]

,title,Alignment
101,Russia. Ukraine. China. Bolton account highlights pattern of Trump welcoming foreign political help.,left


In [678]:
train_df[train_df.tag == 83][['title','Alignment']]

,title,Alignment
83,'We won': DACA recipients overwhelmed by surprise Supreme Court victory over Trump,left


In [755]:
print(test_df[test_df.tag == 94]['title'])
test_vector = test_df[test_df.tag == 94]['paragraph_vector']
sims_negative = model_bow.docvecs.most_similar(negative=test_vector)
sims_negative

94    Bolton book and Supreme Court ruling on ‘dreamers’ amount to twin defeats to Trump
Name: title, dtype: object


[(22, 0.1827678382396698),
 (37, 0.14497077465057373),
 (17, 0.13087917864322662),
 (63, 0.12950745224952698),
 (103, 0.11542965471744537),
 (10, 0.1080268993973732),
 (100, 0.09833711385726929),
 (99, 0.08361626416444778),
 (79, 0.08072808384895325),
 (6, 0.07446075975894928)]

In [701]:
test_df[test_df.tag == 22][['title','Alignment']]

,title,Alignment
22,Liz Harrington: Democrats seek mail-in voting so they can cheat their way to victory in November,right


In [702]:
test_df[test_df.tag == 37][['title','Alignment']]

,title,Alignment
37,"Andrew McCarthy: Defund the police? Here's what Dems, BLM ignoring about crime",right


In [703]:
test_df[test_df.tag == 17][['title','Alignment']]

,title,Alignment
17,Trump says surge in voting by mail his ‘biggest risk’,center


## PV-DMM (Mean)

In [ ]:
news_grid,predictions_y = run_grid_search(dm=1,dm_mean=1,model_path='/Users/tapas/Documents/Repos/news_analysis/model_dmm.bin',stop_words=stop_words_1)

print("Best Cross Validation Score:",news_grid.best_score_)
print("Best Params",news_grid.best_params_)
print('F-Beta Score: ', news_grid.score(test_df[['content_filtered_2']], predictions_y))

prediction_df = test_df
prediction_df['predictions_dmm'] = predictions_y

predictions_y2 = news_grid.predict(out_of_sample_df)
out_of_sample_df['predictions_dmm'] = predictions_y2

prediction_df[['title','Alignment','predictions_dmm']]

'''
Best Cross Validation Score: 0.35812064746847355
Best Params {'doc2vec__dm': 1, 'doc2vec__dm_concat': 0, 'doc2vec__dm_mean': 1, 'doc2vec__epochs': 50, 'doc2vec__min_count': 2, 
'doc2vec__model_path': '/Users/tapas/Documents/Repos/news_analysis/model_dmm.bin', 'doc2vec__sample': 1e-05, 'doc2vec__vector_size': 100,
'doc2vec__window_size': 15}
'''

In [720]:
prediction_df[['title','Alignment','predictions_dmm']]

,title,Alignment,predictions_dmm
26,Defund police in schools? How the movement got momentum after George Floyd's death,center,center
2,"With abortion ruling, Roberts reasserts his role and Supreme Court’s independence",left,right
24,"Trump, at Fox News town hall, says mail-in voting is 'biggest risk' to fair November election, says Dems are 'destroying our country'",right,right
35,"Clyburn says 'nobody is going to defund the police,' calls for 'reimagining' of law enforcement",center,right
90,Steve Levy: DACA decision – Supreme Court's blow to common sense and the rule of law,right,right
6,"Supreme Court strikes down abortion clinic restrictions in Louisiana, a defeat for conservatives",left,center
22,Liz Harrington: Democrats seek mail-in voting so they can cheat their way to victory in November,right,right
62,National security adviser says Trump’s CIA briefer decided not to share Russia bounty intel,right,center
99,Joe Biden needs a China narrative that refutes today’s political angst,left,center
50,People furious over Detroit Zoo’s outdoor mask policy: 'See how many people pass out' in warmer weather,right,center


### Checking similarity vectors in PV-DMM Model

In [763]:
model_dmm = g.Doc2Vec.load("/Users/tapas/Documents/Repos/news_analysis/model_dmm.bin")

def get_paragraph_vector(content):
    vec = model_dmm.infer_vector(content)
    return vec

test_df['paragraph_vector'] = test_df.apply(lambda x: get_paragraph_vector(x['content_filtered_2']),axis=1)

print(test_df[test_df.tag == 90]['title'])
test_vector = test_df[test_df.tag == 90]['paragraph_vector']
sims_positive = model_dmm.docvecs.most_similar(positive=test_vector)
sims_positive

90    Steve Levy: DACA decision – Supreme Court's blow to common sense and the rule of law
Name: title, dtype: object


[(70, 0.2970864176750183),
 (51, 0.20669081807136536),
 (79, 0.17890359461307526),
 (100, 0.1635182797908783),
 (88, 0.10822607576847076),
 (65, 0.09300696849822998),
 (59, 0.08922220766544342),
 (2, 0.07822462916374207),
 (90, 0.07658231258392334),
 (52, 0.07638640701770782)]

- Positive Similarity

In [729]:
test_df[test_df.tag == 51][['title','Alignment']]

,title,Alignment
51,Trump administration asks Supreme Court to strike down Obamacare - The Washington Post,center


In [730]:
test_df[test_df.tag == 70][['title','Alignment']]

,title,Alignment
70,Bolton blasts Trump for denying he was briefed on Russia offering bounties to Taliban to kill US troops,center


In [731]:
test_df[test_df.tag == 79][['title','Alignment']]

,title,Alignment
79,Rep. Greg Walden: Green New Deal would cripple economy – Republicans have better way to fight climate change,right


-  Negative Similarity

In [764]:
print(test_df[test_df.tag == 90]['title'])
test_vector = test_df[test_df.tag == 90]['paragraph_vector']
sims_negative = model_dmm.docvecs.most_similar(negative=test_vector)
sims_negative

90    Steve Levy: DACA decision – Supreme Court's blow to common sense and the rule of law
Name: title, dtype: object


[(75, 0.37485212087631226),
 (30, 0.3317723870277405),
 (112, 0.33068037033081055),
 (58, 0.3229222297668457),
 (45, 0.32242706418037415),
 (46, 0.31581729650497437),
 (73, 0.2997971773147583),
 (53, 0.29486748576164246),
 (71, 0.29220065474510193),
 (23, 0.2845962345600128)]

In [735]:
train_df[train_df.tag == 75][['title','Alignment']]

,title,Alignment
75,"As COVID cases top 3 million, it's past time to end the catastrophic Trump presidency",left


In [736]:
train_df[train_df.tag == 30][['title','Alignment']]

,title,Alignment
30,Power Up: Protesters ‘defund the police’ rallying cry is achieving some progress,left


In [737]:
train_df[train_df.tag == 112][['title','Alignment']]

,title,Alignment
112,Imagine a world with a COVID-19 pandemic and without the Affordable Care Act. Trump does.,left


## PV-DMC (concatnation)

In [775]:
news_grid,predictions_y = run_grid_search(dm=1,dm_concat=1,model_path='/Users/tapas/Documents/Repos/news_analysis/model_dmc.bin',stop_words=stop_words_1)

print("Best Cross Validation Score:",news_grid.best_score_)
print("Best Params",news_grid.best_params_)
print('F-Beta Score: ', news_grid.score(test_df[['content_filtered_2']], predictions_y))

prediction_df = test_df
prediction_df['predictions_dmc'] = predictions_y

predictions_y2 = news_grid.predict(out_of_sample_df)
out_of_sample_df['predictions_dmc'] = predictions_y2

prediction_df[['title','Alignment','predictions_dmc']]

'''
Best Cross Validation Score: 0.4099053030303031
Best Params {'doc2vec__dm': 1, 'doc2vec__dm_concat': 1, 'doc2vec__dm_mean': 0, 'doc2vec__epochs': 50, 'doc2vec__min_count': 2, 'doc2vec__model_path': '/Users/tapas/Documents/Repos/news_analysis/model_dmc.bin', 'doc2vec__sample': 1e-05, 'doc2vec__vector_size': 100, 'doc2vec__window_size': 15}
F-Beta Score:  1.0
'''

100%|██████████| 79/79 [00:00<00:00, 58781.27it/s]


Best Cross Validation Score: 0.39551992032033684
Best Params {'doc2vec__dm': 1, 'doc2vec__dm_concat': 1, 'doc2vec__dm_mean': 0, 'doc2vec__epochs': 50, 'doc2vec__min_count': 2, 'doc2vec__model_path': '/Users/tapas/Documents/Repos/news_analysis/model_dmc.bin', 'doc2vec__sample': 1e-05, 'doc2vec__vector_size': 100, 'doc2vec__window_size': 20}
F-Beta Score:  1.0


"\nBest Cross Validation Score: 0.4099053030303031\nBest Params {'doc2vec__dm': 1, 'doc2vec__dm_concat': 1, 'doc2vec__dm_mean': 0, 'doc2vec__epochs': 50, 'doc2vec__min_count': 2, 'doc2vec__model_path': '/Users/tapas/Documents/Repos/news_analysis/model_dmc.bin', 'doc2vec__sample': 1e-05, 'doc2vec__vector_size': 100, 'doc2vec__window_size': 15}\nF-Beta Score:  1.0\n"

In [777]:
prediction_df[['title','Alignment','predictions_dmc']]

,title,Alignment,predictions_dmc
26,Defund police in schools? How the movement got momentum after George Floyd's death,center,center
2,"With abortion ruling, Roberts reasserts his role and Supreme Court’s independence",left,center
24,"Trump, at Fox News town hall, says mail-in voting is 'biggest risk' to fair November election, says Dems are 'destroying our country'",right,center
35,"Clyburn says 'nobody is going to defund the police,' calls for 'reimagining' of law enforcement",center,right
90,Steve Levy: DACA decision – Supreme Court's blow to common sense and the rule of law,right,center
6,"Supreme Court strikes down abortion clinic restrictions in Louisiana, a defeat for conservatives",left,center
22,Liz Harrington: Democrats seek mail-in voting so they can cheat their way to victory in November,right,center
62,National security adviser says Trump’s CIA briefer decided not to share Russia bounty intel,right,center
99,Joe Biden needs a China narrative that refutes today’s political angst,left,center
50,People furious over Detroit Zoo’s outdoor mask policy: 'See how many people pass out' in warmer weather,right,center


### Checking similarity vectors in PV-DMC Model (concatenation)

In [820]:
model_dmc = g.Doc2Vec.load("/Users/tapas/Documents/Repos/news_analysis/model_dmc.bin")

def get_paragraph_vector(content):
    vec = model_dmc.infer_vector(content)
    return vec

test_df['paragraph_vector'] = test_df.apply(lambda x: get_paragraph_vector(x['content_filtered_2']),axis=1)

print(test_df[test_df.tag == 94]['title'])
test_vector = test_df[test_df.tag == 94]['paragraph_vector']
sims_positive = model_dmc.docvecs.most_similar(positive=test_vector,topn=5)
sims_positive

94    Bolton book and Supreme Court ruling on ‘dreamers’ amount to twin defeats to Trump
Name: title, dtype: object


[(101, 0.40012869238853455),
 (81, 0.3045628070831299),
 (85, 0.3034391701221466),
 (31, 0.2964189648628235),
 (93, 0.2962888181209564)]

- Positive similarity

In [781]:
train_df[train_df.tag == 101][['title','Alignment']]

,title,Alignment
101,Russia. Ukraine. China. Bolton account highlights pattern of Trump welcoming foreign political help.,left


In [782]:
train_df[train_df.tag == 81][['title','Alignment']]

,title,Alignment
81,"On DACA, the Supreme Court tells Trump: You have to follow the rules to change the rules.",left


- Negative Similarity

In [821]:
print(test_df[test_df.tag == 94]['title'])
test_vector = test_df[test_df.tag == 94]['paragraph_vector']
sims_negative = model_dmc.docvecs.most_similar(negative=test_vector,topn = 5)
sims_negative

94    Bolton book and Supreme Court ruling on ‘dreamers’ amount to twin defeats to Trump
Name: title, dtype: object


[(64, 0.2067500650882721),
 (94, 0.18115828931331635),
 (27, 0.15818731486797333),
 (63, 0.1371428668498993),
 (50, 0.1309593915939331)]

In [785]:
test_df[test_df.tag == 64][['title','Alignment']]

,title,Alignment
64,Kayleigh McEnany’s answers on Russia’s bounties and Trump don’t make sense,left


In [790]:
test_df[test_df.tag == 94][['title','Alignment']]

,title,Alignment
94,Bolton book and Supreme Court ruling on ‘dreamers’ amount to twin defeats to Trump,left


In [792]:
print(test_df[test_df.tag == 91]['title'])
test_vector = test_df[test_df.tag == 91]['paragraph_vector']
sims_positive = model_dmc.docvecs.most_similar(positive=test_vector)
sims_positive

91    Cotton blasts Roberts over DACA, invites him to resign, run for office
Name: title, dtype: object


[(92, 0.5574864149093628),
 (45, 0.41880011558532715),
 (89, 0.37993699312210083),
 (88, 0.37665361166000366),
 (93, 0.3760470449924469),
 (87, 0.37565821409225464),
 (85, 0.37419044971466064),
 (19, 0.36854809522628784),
 (108, 0.36389291286468506),
 (30, 0.36339402198791504)]

In [794]:
train_df[train_df.tag == 92][['title','Alignment']]

,title,Alignment
92,Trump calls for 'new justices' after Supreme Court ruling on DACA as conservatives take aim at Roberts,right


In [795]:
train_df[train_df.tag == 45][['title','Alignment']]

,title,Alignment
45,"As support for mask-wearing grows, so do political risks for forgoing them",left


In [796]:
train_df[train_df.tag == 89][['title','Alignment']]

,title,Alignment
89,"Biden calls Supreme Court DACA ruling a ‘victory,’ vows to make program ‘permanent’",right


In [797]:
print(test_df[test_df.tag == 91]['title'])
test_vector = test_df[test_df.tag == 91]['paragraph_vector']
sims_negative = model_dmc.docvecs.most_similar(negative=test_vector)
sims_negative

91    Cotton blasts Roberts over DACA, invites him to resign, run for office
Name: title, dtype: object


[(42, 0.23184587061405182),
 (64, 0.21274758875370026),
 (48, 0.19854673743247986),
 (65, 0.08098895847797394),
 (17, 0.07695845514535904),
 (69, 0.07347670197486877),
 (13, 0.06149737164378166),
 (36, 0.06063408404588699),
 (24, 0.05848829448223114),
 (90, 0.05781293287873268)]

In [799]:
test_df[test_df.tag == 42][['title','Alignment']]

,title,Alignment
42,‘Don’t be a sheep’: Sheriffs rebel against new statewide mask requirements - The Washington Post,left


In [800]:
test_df[test_df.tag == 64][['title','Alignment']]

,title,Alignment
64,Kayleigh McEnany’s answers on Russia’s bounties and Trump don’t make sense,left


## Majority vote prediction from DBOW,PV-DMM and PV-DMC predictions

In [808]:
prediction_df[['title','Alignment','predictions_bow','predictions_dmm','predictions_dmc']].to_excel('prediction_comparison.xlsx')

In [812]:
#print('Accuracy: ', metrics.accuracy_score(y_true=test_df[['bias']], y_pred=predictions_y))
mostCommonVote=[]
for row in prediction_df[['predictions_bow','predictions_dmm','predictions_dmc']].values:
    votes, values = np.unique(row, return_counts=True)
    if np.all(values<=1):
            mostCommonVote.append( row[0] )
    else:
        mostCommonVote.append( votes[np.argmax(values)] )

prediction_df['majority_prediction'] = mostCommonVote

In [817]:
from sklearn.metrics import accuracy_score

print('Accuracy: ', accuracy_score(y_true=prediction_df[['Alignment']], y_pred=prediction_df[['majority_prediction']]))



Accuracy:  0.34285714285714286


### Out-of-sample/ Never before seen accuracy

In [818]:
predictions_y = news_grid.predict(out_of_sample_df)

In [850]:
out_of_sample_df['predictions'] = predictions_y

In [458]:
print('Accuracy: ', accuracy_score(y_true=out_of_sample_df[['Alignment']], y_pred=predictions_y))

Accuracy:  0.5454545454545454


In [851]:
out_of_sample_df['predictions'] = predictions_y

## t-SNE representation of learned vectors

In [825]:
from MulticoreTSNE import MulticoreTSNE as TSNE

tsne_model = TSNE(n_jobs=4,
                  early_exaggeration=4,
                  n_components=2,
                  verbose=1,
                  random_state=2018,
                  n_iter=300)
tsne_d2v = tsne_model.fit_transform(model_dmc.docvecs.vectors_docs)

# Putting the tsne information into sq
tsne_d2v_df = pd.DataFrame(data=tsne_d2v, columns=["x", "y"])
# tsne_tfidf_df.columns = ["x", "y"]
#print(len(tsne_d2v_df))

tsne_d2v_df["source_id"] = df["source_id"].values
tsne_d2v_df["tag"] = df["tag"].values
tsne_d2v_df["title"] = df["title"].values
tsne_d2v_df["Alignment"] = df["Alignment"].values


tsne_d2v_df.to_excel("tsne_viz.xlsx")

In [556]:
pd.set_option('display.max_colwidth', -1)
prediction_df[['title','Alignment','predictions_bow','tag']]

,title,Alignment,predictions_bow,tag
35,"Clyburn says 'nobody is going to defund the police,' calls for 'reimagining' of law enforcement",center,center,35
94,Pompeo: China waging disinformation campaign to drive wedge between US and Europe,center,center,94
70,Bolton blasts Trump for denying he was briefed on Russia offering bounties to Taliban to kill US troops,center,center,70
27,Fact check: Did Adolf Hitler defund the police in Nazi Germany?,left,center,27
22,Liz Harrington: Democrats seek mail-in voting so they can cheat their way to victory in November,right,center,22
62,National security adviser says Trump’s CIA briefer decided not to share Russia bounty intel,right,center,62
17,Trump says surge in voting by mail his ‘biggest risk’,center,right,17
101,Judge blasts Bolton for abandoning classified information review but doubts he can block book publication...,center,center,101
53,Trump administration’s move to end Obamacare amid pandemic reignites political fight,left,right,53
18,Fact-checking the GOP’s ‘satirical’ vote-by-mail video,left,center,18


In [302]:
print(dir(model_dmm.docvecs))

['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__ignoreds', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__numpys', '__recursive_saveloads', '__reduce__', '__reduce_ex__', '__repr__', '__scipys', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_index_to_doctag', '_int_index', '_load_specials', '_save_specials', '_smart_save', 'add', 'closer_than', 'count', 'distance', 'distances', 'doctag_syn0', 'doctag_syn0norm', 'doctags', 'doesnt_match', 'get_vector', 'index2entity', 'index_to_doctag', 'init_sims', 'int_index', 'load', 'mapfile_path', 'max_rawint', 'most_similar', 'most_similar_to_given', 'n_similarity', 'offset2doctag', 'rank', 'save', 'save_word2vec_format', 'similarity', 'similarity_unseen_docs', 'vector_size', 'vectors', 've

In [334]:
#model_dmm.most_similar(['93','37'])
model_dmm.wv.most_similar("republican", topn=20)

[('republicans', 0.6308056712150574),
 ('operative', 0.6204031705856323),
 ('dowless', 0.6086461544036865),
 ('orchestrated', 0.5964101552963257),
 ('substantially', 0.5941324830055237),
 ('mccrae', 0.5941246747970581),
 ('coordinated', 0.5940231680870056),
 ('concluding', 0.5716716051101685),
 ('resourced', 0.5710254907608032),
 ('congressional', 0.5533759593963623),
 ('initiated', 0.5450226068496704),
 ('letting', 0.5266408920288086),
 ('candidate', 0.5251789689064026),
 ('vegas', 0.5183722376823425),
 ('invalidate', 0.5167734622955322),
 ('shattered', 0.5137361288070679),
 ('lawsuit', 0.5105844140052795),
 ('ironically', 0.5105476379394531),
 ('las', 0.5104647874832153),
 ('presidential', 0.5025284290313721)]

## Checking learned keywords for each bias for pictorial representation

In [828]:
prediction_df['content_filtered'] = test_df['content_filtered']
for index,row in prediction_df.iterrows():
    if row['predictions_dmc'] == 'center':
        print(row['content_filtered'])
        wait(1)

['three', 'summers', 'ago', 'about', 'students', 'and', 'activists', 'gathered', 'outside', 'the', 'minneapolis', 'public', 'schools', 'headquarters', 'to', 'demand', 'the', 'school', 'board', 'end', 'contract', 'that', 'placed', 'police', 'officers', 'in', 'schools', 'the', 'officers', 'whose', 'role', 'was', 'to', 'keep', 'schools', 'safe', 'actually', 'made', 'students', 'especially', 'black', 'students', 'feel', 'criminalized', 'activists', 'said', 'but', 'the', 'majority', 'of', 'teachers', 'staff', 'and', 'parents', 'liked', 'officers', 'in', 'schools', 'district', 'wide', 'survey', 'showed', 'at', 'the', 'time', 'the', 'board', 'renewed', 'the', 'three', 'year', 'contract', 'fast', 'forward', 'to', 'june', 'eight', 'days', 'after', 'george', 'floyd', 'died', 'under', 'the', 'knee', 'of', 'minneapolis', 'police', 'officer', 'the', 'school', 'board', 'revisited', 'that', 'contract', 'and', 'ended', 'it', 'the', 'vote', 'on', 'june', 'after', 'an', 'even', 'larger', 'crowd', 'of', 

NameError: name 'wait' is not defined

In [424]:
sum = 0 
for index,row in train_df.iterrows():
    #print(row['Alignment'],len(row['content_filtered']))
    if(row['Alignment'] == 'left'):
        sum+= len(row['content_filtered'])
print(sum)

sum = 0 
for index,row in train_df.iterrows():
    #print(row['Alignment'],len(row['content_filtered']))
    if(row['Alignment'] == 'right'):
        sum+= len(row['content_filtered'])
print(sum)

sum = 0 
for index,row in train_df.iterrows():
    #print(row['Alignment'],len(row['content_filtered']))
    if(row['Alignment'] == 'center'):
        sum+= len(row['content_filtered'])
print(sum)

#pd.reset_option('^display.',silent=True)

train_df[['title','source_id','url','topic','Alignment','article_length','content_full','content_filtered_2']].sort_values(['article_length']).to_excel('temp4.xlsx')

551186
